# In this notebook we look into the performance of L-BFGS-B algorithm.

In [ ]:
from __future__ import division
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy as sc
import pylab as pyl
import time
import cvxpy as cp
from numpy import linalg as Lin
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import style
from sklearn import datasets
import computational_OT

import warnings

warnings.filterwarnings('ignore')


%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
relative_path_to_new_folder = "../Images"
os.makedirs(relative_path_to_new_folder, exist_ok = True)
if not os.path.isdir('../Images/LBFGS_images'):
    os.makedirs('../Images/LBFGS_images')

### Helper Functions

In [ ]:
"""To compute distance matrix"""
def distmat( x, y ):
    return np.sum( x**2, 0 )[:,None] + np.sum( y**2, 0 )[None,:] - 2 * x.transpose().dot( y )

"""To Normalise a vector"""
normalize = lambda a: a/np.sum( a )

"""To Compute P"""
def GetP( u, K, v ):
    return u[:,None] * K * v[None,:]

def plotp( x, col, plt, scale = 200, edgecolors = "k" ):
  return plt.scatter( x[0,:], x[1,:], s = scale, edgecolors = edgecolors, c = col, cmap = 'plasma', linewidths = 2 )

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
def randomsampledata( N ):
  x = []
  y = []
  N = np.sort( N )
  for i in range(len(N)):
    x.append( np.random.rand( 2, N[i] ) - 0.5 )
    theta = 2 * np.pi * np.random.rand( 1, N[i] )
    r = 0.8 + 0.2 * np.random.rand( 1, N[i] )
    y.append( np.vstack( ( np.cos(theta) * r, np.sin(theta) * r ) ) )
  
  return x, y, N

In [ ]:
N = [ 200, 400, 600, 800, 1000 ]
x, y, N = randomsampledata( N )

### I. For fixed epsilon

In [ ]:
BFGSP = []
results_BFGS = []
times_BFGS = []
# BFGS
print("BFGS....")
for i in range(len(N)):

      xi, yi = x[i], y[i]
      #Cost matrix
      C = distmat( xi, yi )
      # a and b
      a = normalize( np.ones( N[i] ) )
      a = a.reshape( a.shape[0], - 1 )
      b = normalize( np.ones( N[i] ) ) 
      b = b.reshape( b.shape[0], - 1 )

      #Epsilon
      epsilon = .06

      #Kernel
      K = np.exp( - C/epsilon )
      
      f,g = a,b
      print( "Doing for ", N[i] )
      print( " |- Iterating" )
      start = time.time()
      Optimizer = computational_OT.L_BFGS_B( K, a, b, f, g, epsilon )
      out = Optimizer._update( maxiter_lbgfs = 100 )
      results_BFGS.append( out )
      end = time.time()
      times_BFGS.append( 1e3 * ( end - start ) )
      print( " |- Computing P")
      print( "" )
      BFGSP.append( GetP( np.exp( out['potential_f']/epsilon ), K, np.exp( out['potential_g']/epsilon ) ) )

     

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 )
plt.title( "$$" )
plt.title( "$||P1 -a||_1$" )

for result in results_BFGS:
  plt.plot( np.asarray(result['error_a']), linewidth = 2 )
plt.yscale( 'log')
plt.legend([ "Population size: "+str(i) for i in N ], loc = "upper right" )
plt.subplot( 2, 1, 2 )

plt.title( "$||P^T 1 -b||_1$" )
for result in results_BFGS:
  plt.plot( np.asarray(result['error_b']), linewidth = 2 )
plt.yscale( 'log' )
plt.legend( [ "Population size: "+str(i) for i in N ], loc = "upper right" )
plt.savefig( "../Images/LBFGS_images/LBFGSconvergence.png" )
plt.show()

print("\n Error plots can increase! The error is not the objective function!")

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.subplot( 2, 1, 1 ),
plt.title( "Objective Function" )
for result in results_BFGS:
  plt.plot( np.asarray(result['objectives']), linewidth = 2 )
plt.legend( [ str(i) for i in N ], loc = "upper right" )
plt.show()

### II. For varying epsilon

In [ ]:
BFGSP = []
results_BFGS = []
epsilons = [ 0.01, 0.05, 0.1, 0.5 ]

# BFGS
print("BFGS....")
N = ( 400, 400 )
for eps in epsilons:

      #Cost matrix
      C = distmat( x[1], y[1] )
      # a and b
      a = normalize( np.ones( N[1] ) )
      a = a.reshape( a.shape[0], - 1 )
      b = normalize( np.ones( N[1] ) )
      b = b.reshape( b.shape[0], - 1 )

      #Kernel
      K = np.exp( - C/eps )
      
      f, g = a, b
      print( "Doing for ",( N[1], N[1] ) )
      print( "\n Epsilon: " +str(eps) )
      print( " |- Iterating" )
      start = time.time()
      Optimizer = computational_OT.L_BFGS_B( K, a, b, f, g, eps )
      out = Optimizer._update( maxiter_lbgfs = 10 )
      results_BFGS.append( out )
      end = time.time()
      print( " |- Computing P")
      print( "" )
      BFGSP.append( GetP( np.exp( out['potential_f']/epsilon ), K, np.exp( out['potential_g']/epsilon ) ) )

     

In [ ]:
plt.figure( figsize = ( 20, 7 ) )
plt.title( "$$" )
plt.title( "$||P1 -a||_1+||P^T 1 -b||_1$" )

for i in range( len(results_BFGS) ):
  error = np.asarray( results_BFGS[i]['error_a'] ) + np.asarray( results_BFGS[i]['error_b'] )
  plt.plot( error, label = 'LBGFS for $\epsilon = $'+ str(epsilons[i]), linewidth = 2 )
plt.yscale( 'log' )
plt.savefig( "../Images/LBFGS_images/LBGFSvaryepsilon.png" )
plt.show()